### imports

In [11]:
import json
import pickle
import re
import numpy as np
import pandas as pd
from Datasets import *
from tqdm.notebook import tqdm
import torch
from torch.nn import DataParallel
from torch.utils.data import DataLoader, RandomSampler
from sklearn.metrics import roc_curve, jaccard_score, accuracy_score
from sklearn.metrics import average_precision_score, coverage_error, label_ranking_average_precision_score, label_ranking_loss, roc_auc_score
import seaborn as sns
from models import PreBuildConverter
from functools import partial
from CKA_torch import CkaLoss
import os
import matplotlib.pylab as plt
import matplotlib
from itertools import combinations
%matplotlib notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

### loading dataset and models 

In [12]:
model_dir = 'work_space/save/CheXpert'

In [13]:
# paren are probes
chexpert_paren = {
    'No_finding': 1,
    'parenchymal': 1,
    'extraparenchymal': 0,
    'limit_out_labels': 1,
    'with_rank':True,
    'with_path':True
}
                  
train, test, in_out, out_out = get_chexpert(**chexpert_paren)
dloader_args = {
        'batch_size': 50,
        'pin_memory': False,
        'num_workers': 0,
        'drop_last': False,
}

train_sampler = RandomSampler(train, replacement=True, num_samples=len(train))
train = DataLoader(train, sampler=train_sampler, **dloader_args)

eval_sampler = RandomSampler(test, replacement=True, num_samples=len(test))
test = DataLoader(test, sampler=eval_sampler, **dloader_args)

in_out_sampler = RandomSampler(in_out, replacement=True, num_samples=len(in_out)//10)
in_out = DataLoader(in_out, sampler=in_out_sampler, **dloader_args)

out_out_sampler = RandomSampler(out_out, replacement=True, num_samples=len(out_out))
out_out = DataLoader(out_out, sampler=out_out_sampler, **dloader_args)

In [115]:
pd.Series(train.dataset.rank_labels.numpy()).map((dict(enumerate(train.dataset.label_names)))).value_counts(normalize=True)

No Finding                    0.413269
Pneumothorax                  0.129036
Pleural Effusion              0.128688
Enlarged Cardiomediastinum    0.097829
Cardiomegaly                  0.097690
Fracture                      0.070589
Support Devices               0.034512
Pleural Other                 0.028389
dtype: float64

In [155]:
pd.Series(train.dataset.rank_labels.numpy()).map((dict(enumerate(['none', 'supp', 'lung', 'cardio', 'fracture'])))).value_counts(normalize=True)

none        0.563119
fracture    0.175824
lung        0.175350
supp        0.047025
cardio      0.038682
dtype: float64

In [156]:
label_ = np.concatenate([train.dataset.labels[:, :2],
train.dataset.labels[:, 2:5].sum(1)[:,None], #Pleural Effusion', 'Pleural Other', 'Pneumothorax'
train.dataset.labels[:, 5:7].sum(1)[:,None], # 'Cardiomegaly', 'Enlarged Cardiomediastinum'
train.dataset.labels[:, 7:]], axis=1)

In [4]:
train.dataset.rank_labels.max().item() + 1

5

### calc emmbeddings

In [18]:
def load_fix(target_path):
    a = torch.load(target_path)#, map_location=lambda storage, loc: storage.cuda())
    fixed_a = {k.split('module.')[-1]: a[k] for k in a}
    torch.save(fixed_a, target_path)

def get_model(target_path, model_type='densenet121'):
    build_model = PreBuildConverter(in_channels=3, out_classes=len(test.dataset.label_names),
                                    add_rank=True, pretrained=False)

    model = build_model.get_by_str(model_type)
    load_fix(target_path)
    model.load_state_dict(torch.load(target_path))
    model.train(mode=False)
    return model

def make_scores_pair(model_paths, loader, train=False, layer='class', rank=False, model_type='densenet121'):
    models = [get_model(model, model_type=model_type).to('cuda') for model in model_paths] # #
    thetas = [[] for i in range(len(model_paths))]
    rank_thetas = [[] for i in range(len(model_paths))]
    labels = []
    rank_labels = []
    cka_class_loss = CkaLoss(models, ['rank_classifier'])
    cka_class_losses = []
    paths = []
    for imgs, label, path in tqdm(loader, total=len(loader)):
        for i,model in enumerate(models):
            theta, rank_theta = model(imgs.cuda())
            rank_thetas[i].append(rank_theta.detach().cpu().numpy())
            thetas[i].append(theta.detach().cpu().numpy())
        if rank:
            label, rank_label = label
            rank_labels.append(rank_label.cpu().numpy())
        labels.append(label.cpu().numpy())
        cka_class_losses.append(cka_class_loss().detach().cpu().numpy().item())
        paths.append(path)
        
    for model in models: del model
    torch.cuda.empty_cache()
    
    paths = np.concatenate(paths)
    labels = np.concatenate(labels)
    rank_labels = [] if not rank else np.concatenate(rank_labels)
    cka_class_losses = np.array(cka_class_losses)
    thetas = [np.concatenate(x) for x in thetas]
    rank_thetas = [np.concatenate(x) for x in rank_thetas]
    
    return thetas, labels, rank_thetas, rank_labels, cka_class_losses, paths

### fuser funcs

In [15]:
def thresh_calibration(thetas, labels):
    model_calibration = []
    for i in range(len(thetas)):
        curr_prob = thetas[i]
        curr_thresh = []
        for j in range(curr_prob.shape[1]):
            fpr, tpr, thresholds = roc_curve(labels[:, j], curr_prob[:, j])
            optimal_idx = np.argmax(tpr - fpr)
            curr_thresh.append(thresholds[optimal_idx])
        model_calibration.append(curr_thresh)
    return model_calibration

def thresh_fuser(mat_a, mat_b, threshhold_a, threshhold_b):
    return (mat_a >= threshhold_a) & (mat_b >= threshhold_b)

def fuser(thetas, labels, calibration_th, n_classes=7, metric=None):
    labels = labels[:, :n_classes]
    fuse_prediction = thresh_fuser(thetas[0], thetas[1], calibration_th[0], calibration_th[1])
    if metric is None:
        return (fuse_prediction == labels).mean(0)
    else:
        return metric(labels, fuse_prediction)

def fuser_ood(thetas, labels, calibration_th, n_classes=7, metric=None):
    labels = labels[:, :n_classes]
    fuse_prediction = thresh_fuser(thetas[0], thetas[1], calibration_th[0], calibration_th[1])
    if metric is None:
        return (fuse_prediction.sum(1) < 1).mean()
    else:
        metric(labels, fuse_prediction)

### model estimators

In [19]:
def get_model_performance(use_model, model_type='densenet121'):
    print(use_model)
    use_model = [os.path.join(use_model, model_path) for model_path in 
               os.listdir(use_model) if 'model' in model_path]
    
    model_in_out_res = make_scores_pair(use_model, in_out, train=True, model_type=model_type)
    model_train = make_scores_pair(use_model, train, train=True, rank=True, model_type=model_type)
    model_test_res = make_scores_pair(use_model, test, train=True, rank=True, model_type=model_type)
    model_out_res = make_scores_pair(use_model, out_out, train=True, model_type=model_type)
    
    model_calibration = thresh_calibration(model_train[0], model_train[1])
    test_res = fuser(model_test_res[0], model_test_res[1], model_calibration)
    print(test_res)
    out_res = fuser(model_out_res[0], model_out_res[1], model_calibration)
    print(out_res)
    in_out_res = fuser(model_in_out_res[0], model_in_out_res[1], model_calibration)
    print(in_out_res)
    ood_out = fuser_ood(model_out_res[0], model_out_res[1], model_calibration)
    print(ood_out)
    ood_in = fuser_ood(model_in_out_res[0], model_in_out_res[1], model_calibration)
    print(ood_in)
    res = dict({'train':model_train, 'test':model_test_res, 'in_out':model_in_out_res, 'out':model_out_res,
                'calibration': model_calibration, 'test_res': test_res, 'in_out_res':in_out_res, 'out_res':out_res,
                'ood_out':ood_out, 'ood_in':ood_in
               })
    return res

In [94]:
out_out.dataset.label_names

array(['No Finding', 'Lung Lesion', 'Lung Opacity', 'Edema',
       'Consolidation', 'Pneumonia', 'Atelectasis', 'Support Devices',
       'Pleural Effusion', 'Pleural Other', 'Pneumothorax',
       'Cardiomegaly', 'Enlarged Cardiomediastinum', 'Fracture'],
      dtype=object)

### load rank models

In [61]:
os.listdir(model_dir +"/Rank")

['2020-07-14_10:09_n=2_densenet121_lr=0.0001_m=_a=0_52_p=False_mean=False_cka=False_pre_pre_layers=-1_-1_pre_steps=100_50_clean-int-ext=1-1-0_limited=1_rank_alpha=0.2_rank_pearson=0',
 '2020-07-14_09:50_n=2_densenet121_lr=0.0001_m=_a=0_52_p=False_mean=False_cka=False_pre_pre_layers=-1_-1_pre_steps=100_50_clean-int-ext=1-1-0_limited=1_rank_alpha=0.01_rank_pearson=0',
 '2020-07-14_10:09_n=2_densenet121_lr=0.0001_m=_a=0_52_p=False_mean=False_cka=False_pre_pre_layers=-1_-1_pre_steps=100_50_clean-int-ext=1-1-0_limited=1_rank_alpha=0.1_rank_pearson=0',
 '2020-07-14_09:50_n=2_densenet121_lr=0.0001_m=_a=0_52_p=False_mean=False_cka=False_pre_pre_layers=-1_-1_pre_steps=100_50_clean-int-ext=1-1-0_limited=1_rank_alpha=0.05_rank_pearson=0']

In [62]:
rank_model_list = [model_dir+'/Rank/'+d for d in os.listdir(model_dir +'/'+"Rank")]

In [73]:
l = [8707, 6651, 2597, 1364, 765, 343, 430]
l = [1375, 3553, 2935, 2142, 2830, 6651, 1371]

In [74]:
pd.Series(np.array(l) / sum(l), index=test.dataset.label_names)

No Finding       0.065925
Lung Lesion      0.170350
Lung Opacity     0.140720
Edema            0.102699
Consolidation    0.135686
Pneumonia        0.318886
Atelectasis      0.065733
dtype: float64

In [72]:
pd.Series(np.array(l) / sum(l), index=test.dataset.label_names)

No Finding       0.417462
Lung Lesion      0.318886
Lung Opacity     0.124515
Edema            0.065398
Consolidation    0.036678
Pneumonia        0.016445
Atelectasis      0.020617
dtype: float64

In [67]:
rank_results = {model:get_model_performance(model) for model in rank_model_list}

work_space/save/CheXpert/Rank/2020-07-14_10:09_n=2_densenet121_lr=0.0001_m=_a=0_52_p=False_mean=False_cka=False_pre_pre_layers=-1_-1_pre_steps=100_50_clean-int-ext=1-1-0_limited=1_rank_alpha=0.2_rank_pearson=0



[0.78703805 0.92564974 0.62945498 0.7914245  0.78638009 0.79888146
 0.73078188]
[0.73742204 0.98515593 0.77488565 0.86952183 0.91197505 0.95629938
 0.864158  ]
[0.92293387 0.9509782  0.50613702 0.68489177 0.70118277 0.83456074
 0.61816559]
0.2998336798336798
0.20851000520717103
work_space/save/CheXpert/Rank/2020-07-14_09:50_n=2_densenet121_lr=0.0001_m=_a=0_52_p=False_mean=False_cka=False_pre_pre_layers=-1_-1_pre_steps=100_50_clean-int-ext=1-1-0_limited=1_rank_alpha=0.01_rank_pearson=0



[0.79668823 0.92652703 0.63351245 0.79032789 0.78265161 0.79613993
 0.73352341]
[0.71488565 0.98873181 0.78706861 0.86665281 0.91775468 0.95575884
 0.86823285]
[0.9164621  0.95335863 0.48902775 0.6808748  0.70743138 0.83396563
 0.61347913]
0.2954261954261954
0.21252696570705942
work_space/save/CheXpert/Rank/2020-07-14_10:09_n=2_densenet121_lr=0.0001_m=_a=0_52_p=False_mean=False_cka=False_pre_pre_layers=-1_-1_pre_steps=100_50_clean-int-ext=1-1-0_limited=1_rank_alpha=0.1_rank_pearson=0



[0.78495449 0.92959754 0.63186753 0.79230179 0.78780568 0.80107468
 0.73286545]
[0.72336798 0.98939709 0.77858628 0.86977131 0.92253638 0.9604158
 0.88390852]
[0.92509113 0.94971361 0.49408614 0.68861117 0.7028937  0.84021424
 0.62590196]
0.30735966735966735
0.22219742616975377
work_space/save/CheXpert/Rank/2020-07-14_09:50_n=2_densenet121_lr=0.0001_m=_a=0_52_p=False_mean=False_cka=False_pre_pre_layers=-1_-1_pre_steps=100_50_clean-int-ext=1-1-0_limited=1_rank_alpha=0.05_rank_pearson=0



[0.78681873 0.92608839 0.62561684 0.78769602 0.79274043 0.80513214
 0.74416054]
[0.74054054 0.98731809 0.77580042 0.8770894  0.92195426 0.96203742
 0.86070686]
[0.92933125 0.95015993 0.5018969  0.68898311 0.71100201 0.83798259
 0.62032284]
0.31654885654885656
0.21691586699397455


In [68]:
pickle.dump(rank_results, open('rank_res', 'wb'))
#rank_results = pickle.load(open('rank_res', 'rb'))

In [22]:
def get_agreemnet(res):
    res_1 = []
    emba, embb = res['train'][2]
    res_1.append((np.argmax(emba, 1) == np.argmax(embb, 1)).mean())
    emba, embb = res['test'][2]
    res_1.append((np.argmax(emba, 1) == np.argmax(embb, 1)).mean())
    emba, embb = res['out'][2]
    res_1.append((np.argmax(emba, 1) == np.argmax(embb, 1)).mean())
    emba, embb = res['in_out'][2]
    res_1.append((np.argmax(emba, 1) == np.argmax(embb, 1)).mean())
    return res_1

In [70]:
for model in rank_results:
    print(model)
    get_agreemnet(rank_results[model])

work_space/save/CheXpert/Rank/2020-07-14_10:09_n=2_densenet121_lr=0.0001_m=_a=0_52_p=False_mean=False_cka=False_pre_pre_layers=-1_-1_pre_steps=100_50_clean-int-ext=1-1-0_limited=1_rank_alpha=0.2_rank_pearson=0


[0.931821450831855,
 0.47779361772124135,
 0.4414968814968815,
 0.4040764710258127]

work_space/save/CheXpert/Rank/2020-07-14_09:50_n=2_densenet121_lr=0.0001_m=_a=0_52_p=False_mean=False_cka=False_pre_pre_layers=-1_-1_pre_steps=100_50_clean-int-ext=1-1-0_limited=1_rank_alpha=0.01_rank_pearson=0


[0.8226494702018508,
 0.7821033008005264,
 0.8046569646569647,
 0.6803540876292494]

work_space/save/CheXpert/Rank/2020-07-14_10:09_n=2_densenet121_lr=0.0001_m=_a=0_52_p=False_mean=False_cka=False_pre_pre_layers=-1_-1_pre_steps=100_50_clean-int-ext=1-1-0_limited=1_rank_alpha=0.1_rank_pearson=0


[0.947499640408496,
 0.5018094089264173,
 0.46074844074844074,
 0.4112177341367254]

work_space/save/CheXpert/Rank/2020-07-14_09:50_n=2_densenet121_lr=0.0001_m=_a=0_52_p=False_mean=False_cka=False_pre_pre_layers=-1_-1_pre_steps=100_50_clean-int-ext=1-1-0_limited=1_rank_alpha=0.05_rank_pearson=0


[0.7803135637915328,
 0.5173812918083123,
 0.47442827442827445,
 0.4278806813955218]

In [91]:
ind = 2
res_2 = []
emba, embb = rank_results[list(rank_results.keys())[ind]]['test'][2]
(np.argmax((emba + embb) / 2, 1) == rank_results[list(rank_results.keys())[ind]]['test'][3]).mean()

0.40092115363526704

In [88]:
rank_results[list(rank_results.keys())[ind]]['test'][3]

array([0, 6, 0, ..., 5, 4, 4])

### vgg trail

In [7]:
rank_model_list = [model_dir+'/Rank/'+d for d in os.listdir(model_dir +'/'+"Rank") if 'vgg' in d]

In [8]:
rank_model_list

['work_space/save/CheXpert/Rank/2020-07-15_09:01_n=2_vgg16_lr=0.0001_m=_a=0_32_p=False_mean=False_cka=False_pre_pre_layers=-1_pre_steps=50_clean-int-ext=1-1-0_limited=1_rank_alpha=0.2_rank_pearson=0']

In [20]:
vgg_results = {model:get_model_performance(model, 'vgg16') for model in rank_model_list}

work_space/save/CheXpert/Rank/2020-07-15_09:01_n=2_vgg16_lr=0.0001_m=_a=0_32_p=False_mean=False_cka=False_pre_pre_layers=-1_pre_steps=50_clean-int-ext=1-1-0_limited=1_rank_alpha=0.2_rank_pearson=0



[0.774756   0.65358044 0.64272398 0.74383156 0.596118   0.68329861
 0.67046825]
[0.60182952 0.61193347 0.48740125 0.68074844 0.54540541 0.71683992
 0.5249896 ]
[0.90188202 0.79297776 0.65506211 0.62285204 0.38919884 0.55188574
 0.55106747]
0.004241164241164241
0.0026779736665922785


In [23]:
for model in vgg_results:
    print(model)
    get_agreemnet(vgg_results[model])

work_space/save/CheXpert/Rank/2020-07-15_09:01_n=2_vgg16_lr=0.0001_m=_a=0_32_p=False_mean=False_cka=False_pre_pre_layers=-1_pre_steps=50_clean-int-ext=1-1-0_limited=1_rank_alpha=0.2_rank_pearson=0


[0.9201706861005897,
 0.9152319333260226,
 0.9088149688149688,
 0.8861861191698281]